In [1]:
import pandas as pd

In [3]:
grid = pd.DataFrame(columns = ["lat", "long", "population"])

GET GRIDS

In [8]:
import sys
sys.path.append("../tools")
from grids import pxtolat,pxtolong

In [9]:
with open('./gpw_pop.asc', 'r') as f:
        for i in range(6):
            f.readline() # skip pre data

        for rn in range(4320):
            k = f.readline().split(' ')
            for c in range(8640):
                cell_val = float(k[c]) # read population in grid

                if cell_val < 0: # skip pixel if no population
                    continue
                grid.append([pxtolat(rn),pxtolong(c),cell_val])

C:\Users\vogeld2\AppData\Local\Temp\ipykernel_20248\1666486226.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid.append([pxtolat(rn),pxtolong(c)])
C:\Users\vogeld2\AppData\Local\Temp\ipykernel_20248\1666486226.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid.append([pxtolat(rn),pxtolong(c)])
C:\Users\vogeld2\AppData\Local\Temp\ipykernel_20248\1666486226.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid.append([pxtolat(rn),pxtolong(c)])
C:\Users\vogeld2\AppData\Local\Temp\ipykernel_20248\1666486226.py:8: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  grid.append([pxtolat(rn),pxtolong(c)])
C:\Users\vogeld2\AppData\Loc

POPULATION

In [ ]:
pd.read_table()

EARTHQUAKES

TSUNAMI

HURRICANES

WIND

SOLAR